In [1]:
import requests
import pandas as pd
pd.options.display.max_columns = 100
import numpy as np

In [49]:
r = requests.get('http://fcast.us-east-1.espncdn.com/FastcastService/pubsub/profiles/12000/topic/event-topevents-espncricinfo-gb-en/message/1006375/checkpoint').json()

tours = pd.DataFrame(r['sports'][0]['leagues'])[['abbreviation', 'isTournament', 'name']]
tours.columns = ['abbreviation', 'isTournament', 'tour_name']

matches = pd.DataFrame()
for match in r['sports'][0]['leagues']:
    matches = matches.append(pd.DataFrame(match['events']) \
    [['date', 'description', 'endDate', 'eventType', 'name', 'shortName', 'title']]).reset_index(drop=True)

matches2 = pd.DataFrame()
for match in r['sports'][0]['leagues']:
    matches2 = matches2.append(pd.DataFrame(match['events'][0]['fullStatus']).loc[['description']] \
                               [['dayNumber', 'period', 'summary', 'type']]).reset_index(drop=True)

home_teams = []; away_teams = []
is_home_national = []; is_away_national = []
batting_first_team = []; winners = []
innings1 = []; innings2 = []
link = []; ground = []

for match in r['sports'][0]['leagues']:
    home_teams.append(match['events'][0]['competitors'][0]['displayName'])
    away_teams.append(match['events'][0]['competitors'][1]['displayName'])   
    link.append(match['events'][0]['link'])
    ground.append(match['events'][0]['location'])
    
    if match['events'][0]['competitors'][0]['isNational']:
        is_home_national.append(1)
    else:
        is_home_national.append(0)
        
    if match['events'][0]['competitors'][1]['isNational']:
        is_away_national.append(1)
    else:
        is_away_national.append(0)
    
    if match['events'][0]['competitors'][0]['winner']:
        winners.append(match['events'][0]['competitors'][0]['displayName'])
    elif match['events'][0]['competitors'][1]['winner']:
        winners.append(match['events'][0]['competitors'][1]['displayName'])
    else:
        winners.append(np.nan)
    
    if match['events'][0]['competitors'][0]['order'] == 1:
        batting_first_team.append(match['events'][0]['competitors'][0]['displayName'])
    else:
        batting_first_team.append(match['events'][0]['competitors'][1]['displayName'])
        
    if match['events'][0]['competitors'][0]['order'] == 1:
        innings1.append(match['events'][0]['competitors'][0]['score'])
        innings2.append(match['events'][0]['competitors'][1]['score'])
    else:
        innings1.append(match['events'][0]['competitors'][1]['score'])
        innings2.append(match['events'][0]['competitors'][0]['score'])

teams = pd.DataFrame(zip(home_teams, away_teams, is_home_national, is_away_national,
                         batting_first_team, winners, innings1, innings2, ground, link), 
                     columns = ['home_team', 'away_team', 'is_home_national', 'is_away_national',
                                'batting_first_team', 'winner', 'innings1', 'innings2', 'ground', 'scorecard_link'])

df = matches2.join(matches).join(tours).join(teams)
df.columns = ['current_day', 'current_innings', 'match_summary', 'is_finished', 'full_date', 'description', 'end_date', 'match_format',
              'match_name', 'match_short_name', 'match_title', 'tour_abbrev', 'is_tournament', 'tour_name', 
              'home_team', 'away_team', 'is_home_national', 'is_away_national', 'batting_first_team', 
              'winner', 'innings1', 'innings2', 'ground', 'scorecard_link']

df['match_start_date'] = df.full_date.apply(pd.to_datetime).dt.date
df['match_start_time'] = df.full_date.apply(pd.to_datetime).dt.time
df['match_end_date'] = df.end_date.apply(pd.to_datetime).dt.date

In [50]:
df

,current_day,current_innings,match_summary,is_finished,full_date,description,end_date,match_format,match_name,match_short_name,match_title,tour_abbrev,is_tournament,tour_name,home_team,away_team,is_home_national,is_away_national,batting_first_team,winner,innings1,innings2,ground,scorecard_link,match_start_date,match_start_time,match_end_date
0,1,2,WI Women won by 6 wickets (with 66 balls remai...,Result,2017-10-11T13:30:00Z,"1st ODI, ICC Women's Championship at Tarouba, ...",2017-10-12T23:59:00Z,ODI,West Indies Women v Sri Lanka Women,WI-W v SL-W,1st ODI,ICC Women's Championship,True,ICC Women's Championship,West Indies Women,Sri Lanka Women,0,0,Sri Lanka Women,West Indies Women,136,"138/4 (39/50 ov, target 137)","Brian Lara Stadium, Tarouba, Trinidad",http://www.espncricinfo.com/series/8674/scorec...,2017-10-11,13:30:00,2017-10-12
1,1,1,Bangladesh won the toss and elected to bat,Live,2017-10-12T08:00:00Z,"Tour Match, Bangladesh tour of South Africa at...",2017-10-12T23:59:00Z,ODI,Cricket South Africa Invitation XI v Bangladesh,SA-XI v BDESH,Tour Match,Bangladesh tour of SA 2017-18,False,Bangladesh tour of South Africa 2017/18,Cricket South Africa Invitation XI,Bangladesh,0,1,Bangladesh,NaN,105/4 (21.1/50 ov),,"Mangaung Oval, Bloemfontein",http://www.espncricinfo.com/series/10904/game/...,2017-10-12,08:00:00,2017-10-12
2,2,2,Bangladesh A trail by 91 runs with 7 wickets r...,Live,2017-10-11T03:30:00Z,"Only unofficial Test, Ireland A tour of Bangla...",2017-10-14T23:59:00Z,Test,Bangladesh A v Ireland A,BdshA v IrelA,Only unofficial Test,IRE-A in BDESH 2017-18,False,Ireland A tour of Bangladesh 2017/18,Bangladesh A,Ireland A,0,0,Ireland A,NaN,255,164/3 (54 ov),Sylhet International Cricket Stadium,http://www.espncricinfo.com/series/18096/game/...,2017-10-11,03:30:00,2017-10-14
